# Understanding TensorRT - Convert Graph

In [ ]:
## Convert the Model
### Step 1 - convert the checkpoint to TensorFlow Lite

See TrainModel_Step3_TrainingJob.ipynb

In [ ]:
#### Convert (Trained) Model Checkpoint to a tflite Model


ALSO NOTE:  the script is named convert_checkpoint_to_edge_tflite.sh  
Well... the name is no longer totally accurate
- I took this from the original Coral TPU tutorial
- Another step is required for compiling for the EdgeTPU (not really relevant here since we are confined to AWS where there is no TPU -- so we skip that stuff)
- And, I added a step that converts the checkpoint to a TENSORFLOW frozen graph 
  - note that this generates a frozen_inference_graph.pb
  - it ALSO generates a saved model graph.pb  
  THESE (frozen graph & saved model) are NOT the SAME!!  
  
https://stackoverflow.com/questions/46547319/error-when-parsing-graph-def-from-string

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/exporting_models.md

In [ ]:
! ls code/model

In [ ]:
# WAKE UP - make sure the checkpoint num == hyperparameters/num_train_steps
# convert checkpoint is a task script - located in the tasks/ directory
os.chdir("tasks")  
! ./convert_checkpoint_to_edgetpu_tflite.sh --checkpoint_num {NUM_TRAINING_STEPS} --pipeline_config {PIPELINE_CONFIG}


In [ ]:
# https://www.tensorflow.org/lite/convert/python_api#using_the_interpreter_from_a_model_file_

# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model('code/model')
tflite_model = converter.convert()